In [1]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Load dataset
digits = load_digits()
X = digits.images / 16.0
y = digits.target.reshape(-1, 1)

# One-hot encode labels
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y)

# Flatten (8x8 → 64 features)
X = X.reshape(len(X), -1)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)

In [3]:
# Bigger model (~19k params)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation="relu", input_shape=(64,)),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [4]:
history = model.fit(X_train, y_train, epochs=30, batch_size=32,
                    validation_split=0.1, verbose=1)

loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {acc:.4f}")

model.summary()

Epoch 1/30
41/41 [==============================] - 3s 8ms/step - loss: 1.9602 - accuracy: 0.4532 - val_loss: 1.4083 - val_accuracy: 0.7431
Epoch 2/30
41/41 [==============================] - 0s 5ms/step - loss: 0.9244 - accuracy: 0.8175 - val_loss: 0.5352 - val_accuracy: 0.8819
Epoch 3/30
41/41 [==============================] - 0s 4ms/step - loss: 0.3605 - accuracy: 0.9258 - val_loss: 0.3199 - val_accuracy: 0.9097
Epoch 4/30
41/41 [==============================] - 0s 4ms/step - loss: 0.2185 - accuracy: 0.9559 - val_loss: 0.2213 - val_accuracy: 0.9306
Epoch 5/30
41/41 [==============================] - 0s 4ms/step - loss: 0.1657 - accuracy: 0.9598 - val_loss: 0.1839 - val_accuracy: 0.9444
Epoch 6/30
41/41 [==============================] - 0s 4ms/step - loss: 0.1293 - accuracy: 0.9683 - val_loss: 0.1394 - val_accuracy: 0.9583
Epoch 7/30
41/41 [==============================] - 0s 4ms/step - loss: 0.1073 - accuracy: 0.9722 - val_loss: 0.1607 - val_accuracy: 0.9375
Epoch 8/30
41/41 [==

In [6]:
# Convert to float32 TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("digits_model_19k.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\kukil\AppData\Local\Temp\tmpquhb08jf\assets


INFO:tensorflow:Assets written to: C:\Users\kukil\AppData\Local\Temp\tmpquhb08jf\assets


In [7]:
# Quantization (dynamic range)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

with open("digits_model_quant_19k.tflite", "wb") as f:
    f.write(tflite_quant_model)

INFO:tensorflow:Assets written to: C:\Users\kukil\AppData\Local\Temp\tmpwdzppm9u\assets


INFO:tensorflow:Assets written to: C:\Users\kukil\AppData\Local\Temp\tmpwdzppm9u\assets


In [9]:
!xxd -i digits_model_quant_19k.tflite > digits_model_quant_19k.h